In [1]:
!pip install pandas dash
import pandas as pd
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB 640.0 kB/s eta 0:00:16
    --------------------------------------- 0.1/10.2 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.4/10.2 MB 2.9 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/10.2 MB 3.6 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.2 MB 3.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/10.2 MB 3.8 MB/s eta 0:00:03
   ----- ---------------------------------- 1.3/10.2 MB 4.2 MB/s eta 0:00:03
   ----- ---------------------------------- 1.5/10.2 MB 4.2 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/10.2 MB 4.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/10.2 MB 4.4 MB/s eta 0:00:02
   ------- -------------------------------- 2.0/10.2 MB 4.4 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.2 MB 4.1 MB/s eta 0:00:02
   -

In [2]:
#download dataset as spacex_launch_dash.csv
import urllib.request

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
filename = "spacex_launch_dash.csv"

urllib.request.urlretrieve(url, filename)


('spacex_launch_dash.csv', <http.client.HTTPMessage at 0x2c7fda3ae50>)

In [3]:
#Download a skeleton Dash app to be completed in this lab:


url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
filenamedash = "spacex_dash_app.py"

urllib.request.urlretrieve(url, filenamedash)


('spacex_dash_app.py', <http.client.HTTPMessage at 0x2c7fda45890>)

In [4]:
%run spacex_dash_app.py



C:\Users\shred\Desktop\capstone\spacex_dash_app.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\shred\Desktop\capstone\spacex_dash_app.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [5]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
min_payload = int(spacex_df['Payload Mass (kg)'].min())
max_payload = int(spacex_df['Payload Mass (kg)'].max())

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Task 1: Add a dropdown list to enable Launch Site selection
                                dcc.Dropdown(id='site-dropdown',
                                             options=[
                                                 {'label': 'All Sites', 'value': 'ALL'},
                                                 {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                 {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                 {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                 {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                             ],
                                             value='ALL',
                                             placeholder="Select a Launch Site here",
                                             searchable=True
                                             ),
                                html.Br(),

                                # Task 2: Add a pie chart to show the total successful launches count for all sites
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # Task 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=min_payload, max=max_payload, step=1000,
                                                marks={i: str(i) for i in range(min_payload, max_payload+1, 1000)},
                                                value=[min_payload, max_payload]),

                                # Task 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# Task 2: Add a callback function for site-dropdown as input, success-pie-chart as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launches for All Sites')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        counts = filtered_df['class'].value_counts()
        fig = px.pie(names=counts.index, values=counts.values, title=f'Success vs. Failure for {entered_site}')
    return fig

# Task 4: Add a callback function for site-dropdown and payload-slider as inputs, success-payload-scatter-chart as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
               Input(component_id="payload-slider", component_property="value")])
def get_scatter_chart(entered_site, payload_range):
    if entered_site == 'ALL':
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1])]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title='Payload vs. Launch Outcome for All Sites')
    else:
        filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= payload_range[0]) & 
                                (spacex_df['Payload Mass (kg)'] <= payload_range[1]) &
                                (spacex_df['Launch Site'] == entered_site)]
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version Category',
                         title=f'Payload vs. Launch Outcome for {entered_site}')
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
